# DBSCAN (밀도 기반 클러스터링)

## DBSCAN: Density-based spetial clustering of applications with noise

기존의 K-Means, Hierarchical 클러스터링 -> 군집간의 거리를 이용하는 클러스터링   
DBSCAN은 점이 얼마나 세밀하게 몰료있는지 (밀도가 높은지)롤 보고 군집을 인식한다.   